In [ ]:
import tkinter
from tkinter import ttk
from socket import AF_INET, socket, SOCK_STREAM
from threading import Thread
import pyaudio
import wave
import os
import sys

# Global variables for recording
frames = []
stream = None
recording = False

def receive():
    """Handles receiving of messages."""
    while True:
        try:
            msg = client_socket.recv(BUFSIZ).decode("utf8")
            if msg:
                update_message_list(msg)  # Update the message list in the GUI
        except OSError:  # Possibly client has left the chat.
            break

def update_message_list(msg):
    """Updates the message list in the GUI."""
    msg_list.insert(tkinter.END, msg + '\n')
    msg_list.see(tkinter.END)  # Automatically scroll to the end

def send(event=None):  # event is passed by binders.
    """Handles sending of messages."""
    msg = my_msg.get()
    my_msg.set("")  # Clears input field.
    client_socket.send(bytes("text", "utf8"))
    client_socket.send(bytes(msg, "utf8"))
    update_message_list("You: " + msg)  # Add the client's message to the chat window
    if msg == "{quit}":
        client_socket.close()
        top.destroy()  # Close the GUI window

def record():
    """Records voice message."""
    global frames
    global stream
    global recording
    recording = True
    frames = []
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100
    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    update_message_list("Recording...")

    while recording:
        data = stream.read(CHUNK)
        frames.append(data)
        top.update()  # Update the GUI to prevent it from becoming unresponsive

def pause_and_send():
    """Pauses recording and sends the recorded audio to the server."""
    global recording
    global stream
    recording = False
    update_message_list("Recording paused.")
    if stream:  # Check if stream is defined
        stream.stop_stream()
        stream.close()

    # Save the recorded audio to a WAV file
    file_name = "recorded_message.wav"
    wave_file = wave.open(file_name, 'wb')
    wave_file.setnchannels(1)
    wave_file.setsampwidth(2)
    wave_file.setframerate(44100)
    wave_file.writeframes(b''.join(frames))
    wave_file.close()

    # Send the audio file to the server
    try:
        with open(file_name, "rb") as audio_file:
            audio_data = audio_file.read()
            client_socket.send(b"voice")  # Send a message type indicator for voice message
            client_socket.send(bytes(str(len(audio_data)), "utf8"))  # Send the file size
            client_socket.send(audio_data)
            update_message_list("Voice message sent.")
    except Exception as e:
        print(f"Error sending voice message: {e}")

    # Delete the temporary audio file
    os.remove(file_name)

# Function to handle closing of the GUI window
def on_closing():
    """This function is to be called when the window is closed."""
    my_msg.set("{quit}")
    send()

# Setup the GUI
top = tkinter.Tk()
top.title("Chat Client")

# Define a custom style for the theme
style = ttk.Style()
style.theme_use('clam')  # Use the 'clam' theme for a dark theme

# Create a frame for the message list
messages_frame = tkinter.Frame(top)
messages_frame.pack(expand=True, fill=tkinter.BOTH)

# Create a scrollbar for the message list
scrollbar = tkinter.Scrollbar(messages_frame)
scrollbar.pack(side=tkinter.RIGHT, fill=tkinter.Y)

# Create a text widget for displaying messages
msg_list = tkinter.Text(messages_frame, height=15, width=50, wrap=tkinter.WORD, yscrollcommand=scrollbar.set, bg='grey', fg='black', font=('Helvetica', 12))
msg_list.pack(side=tkinter.LEFT, expand=True, fill=tkinter.BOTH)

# Configure the scrollbar to work with the text widget
scrollbar.config(command=msg_list.yview)

# Create a frame for the input field and buttons
input_frame = tkinter.Frame(top)
input_frame.pack(expand=True, fill=tkinter.X)

# Create an entry field for typing messages
my_msg = tkinter.StringVar()  # For the messages to be sent.
my_msg.set("Type your messages here.")
entry_field = tkinter.Entry(input_frame, textvariable=my_msg, bg='grey', fg='black', font=('Helvetica', 12))
entry_field.pack(side=tkinter.LEFT, expand=True, fill=tkinter.X)

# Create a send button
send_button = ttk.Button(input_frame, text="Send", command=send)
send_button.pack(side=tkinter.RIGHT)

# Create a record button
record_button = ttk.Button(input_frame, text="Record", command=lambda: Thread(target=record).start())
record_button.pack(side=tkinter.RIGHT)

# Create a pause button
pause_button = ttk.Button(input_frame, text="Pause", command=pause_and_send)
pause_button.pack(side=tkinter.RIGHT)

# Bind the return key to the send function
top.bind("<Return>", send)

# Configure the close button to call the on_closing function
top.protocol("WM_DELETE_WINDOW", on_closing)

# Connect to the server
HOST = 'localhost'  # Server's IP address or domain name
PORT = 33000  # Server's port number
BUFSIZ = 1024
ADDR = (HOST, PORT)

try:
    client_socket = socket(AF_INET, SOCK_STREAM)
    client_socket.connect(ADDR)
except Exception as e:
    print(f"Unable to connect to the server: {e}")
    sys.exit(1)

# Start the thread for receiving messages
receive_thread = Thread(target=receive)
receive_thread.start()

# Set the size of the window
top.geometry("600x400")

# Start the GUI event loop
tkinter.mainloop()


In [19]:
!pip install pyaudio pillow


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pyaudio from https://files.pythonhosted.org/packages/82/d8/f043c854aad450a76e476b0cf9cda1956419e1dacf1062eb9df3c0055abe/PyAudio-0.2.14-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/164.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/164.1 kB ? eta -:--:--
   ---- ---------------------------------- 20.5/164.1 kB 222.6 kB/s eta 0:00:01
   ---- ---------------------------------- 20.5/164.1 kB 222.6 kB/s eta 0:00:01
   ------------ -------------------------- 51.2/164.1 kB 292.6 kB/s eta 0:00:01
   ------------------- ------------------- 81.9/164.1 kB 383.3 kB/s eta 0:00:01
   -------------------------------------- 164.1/164.1 kB 615.3 kB/s eta 0:00:00
